In [1]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('/content/drive/MyDrive/Datasets/Movie_Compiled.csv')
data.head()

,Rating,Date,Title,Review
0,,12-Apr-12,An Unstoppable Force of Awesomeness! All Hail ...,I have got to tell anyone who will listen that...
1,,19-Apr-12,They balanced everything masterfully. As Good ...,"If you're a fan of epic adventure movies, then..."
2,10,20-Apr-12,The Avengers assembled flawlessly,I'm sorry to say The Avengers isn't a good mov...
3,9,16-Apr-12,LA Review of 'The Avengers' (No spoilers),LA Review of 'The Avengers' (No spoilers)\r\n\...
4,10,02-Jul-17,Bad-ass descent-amazing superhero MCU action f...,Marvel's The Avengers (2012) is an awesome des...


In [4]:
# Document matrix will be very sparse and we cant work with it directly as ML m odels will give very bad results.tHUS WE USE truncated svd to create an 
# approximation to this sparse matrix.Truncated SVD gives us the matrix based on the number of dimensions that i want to extract.

In [5]:
#This is a sparse matrix
#This is rating prediction for those that are missing
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import cohen_kappa_score

In [6]:
data_1 = data[~data['Review'].isnull()]
data_1.shape

(4402, 4)

In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4402 entries, 0 to 4501
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rating  4075 non-null   object
 1   Date    4402 non-null   object
 2   Title   4402 non-null   object
 3   Review  4402 non-null   object
dtypes: object(4)
memory usage: 172.0+ KB


In [8]:
data_1.Rating.value_counts()

10    1143
1      719
9      428
8      301
7      288
6      252
5      232
3      225
2      218
4      202
        67
Name: Rating, dtype: int64

In [10]:
data_2=data_1.replace(' ',np.nan)
data_2.head()

,Rating,Date,Title,Review
0,NaN,12-Apr-12,An Unstoppable Force of Awesomeness! All Hail ...,I have got to tell anyone who will listen that...
1,NaN,19-Apr-12,They balanced everything masterfully. As Good ...,"If you're a fan of epic adventure movies, then..."
2,10,20-Apr-12,The Avengers assembled flawlessly,I'm sorry to say The Avengers isn't a good mov...
3,9,16-Apr-12,LA Review of 'The Avengers' (No spoilers),LA Review of 'The Avengers' (No spoilers)\r\n\...
4,10,02-Jul-17,Bad-ass descent-amazing superhero MCU action f...,Marvel's The Avengers (2012) is an awesome des...


In [ ]:
# If no of spaces were not uniform then will need to use / + to replace all the spaces and then  replace it with np.nan

In [11]:
data_2["Rating"]=data_2["Rating"].astype(float)
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4402 entries, 0 to 4501
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rating  4008 non-null   float64
 1   Date    4402 non-null   object 
 2   Title   4402 non-null   object 
 3   Review  4402 non-null   object 
dtypes: float64(1), object(3)
memory usage: 172.0+ KB


In [12]:
train_data=data_2[~data_2['Rating'].isnull()]
test_data=data_2[data_2['Rating'].isnull()]

In [13]:
(train_data.shape,test_data.shape)

((4008, 4), (394, 4))

In [14]:
train_data

,Rating,Date,Title,Review
2,10.0,20-Apr-12,The Avengers assembled flawlessly,I'm sorry to say The Avengers isn't a good mov...
3,9.0,16-Apr-12,LA Review of 'The Avengers' (No spoilers),LA Review of 'The Avengers' (No spoilers)\r\n\...
4,10.0,02-Jul-17,Bad-ass descent-amazing superhero MCU action f...,Marvel's The Avengers (2012) is an awesome des...
6,10.0,15-Apr-12,A Comic Book Triumph!,I was lucky enough to be included in the group...
8,9.0,14-Apr-12,Nothing Short of Amazing,I just saw the early screening for San Diego t...
...,...,...,...,...
4496,10.0,17-Aug-05,A little story,After an attack by a pack of saber-toothed cat...
4498,9.0,25-Jun-04,Great characters in a fun adventure!,"""Ice Age"" gives us a sabertooth tiger, a sloth..."
4499,10.0,01-Jun-04,"""We are definitely the weirdest tribe I've eve...","Ice Age was a very entertaining movie, with br..."
4500,9.0,25-Apr-04,This was a cool movie,"Ice Age, thats a good title. I was very confus..."


In [15]:
x_train, x_val, y_train, y_val = train_test_split(train_data.iloc[:,1:], train_data.iloc[:,0], test_size = 0.4, random_state= 123456)

In [16]:
from wordcloud import STOPWORDS

In [17]:
tfidf= TfidfVectorizer(stop_words=STOPWORDS,token_pattern="[a-z']+",max_df=0.55,min_df=10)

In [18]:
tfidf_mat_train=tfidf.fit_transform(x_train['Review'])
tfidf_mat_val=tfidf.transform(x_val['Review'])
tfidf_mat_test=tfidf.transform(test_data['Review'])

In [ ]:
# care dont fit transform on the val and test otherwise the model will be retrained on the new data.

In [19]:
tfidf_mat_train.shape

(2404, 3657)

In [42]:
x_train

,Date,Title,Review
3976,05-May-18,stupid,....he will win 100% Oscar for the worst scena...
2670,27-12-2008 00:00,The perfect movie.,The Godfather is probably the most perfect mov...
672,26-01-2011 00:00,"Good movie, but very overrated",One is a rate to balance properly the fanatic ...
908,04-12-2010 00:00,"Overrated because it makes people ""feel"" smart...",Christopher Nolan has done some great work.\r\...
4359,19-May-05,"Very Funny Movie. A lot better than ""Robots""",This is an extremely funny movie. Although som...
...,...,...,...
4092,11-May-18,Really? This was the culmination of 10 years o...,Completely disappointing. A few mildly witty o...
3511,22-May-18,Entertainment,"If you are looking to be entertained great, bu..."
523,13-12-2010 00:00,WAAAAY Overrated,I'll try to make this as short and painless as...
3174,12-Jan-16,Every space clichÃ© you've ever seen for the p...,Is it my fault Matthew MacHaunecatechity has o...


In [20]:
lr_model_1=LogisticRegression()
lr_model_1.fit(X=tfidf_mat_train,y=y_train)

LogisticRegression()

In [21]:
pred_val_lr=lr_model_1.predict(tfidf_mat_val)
cohen_kappa_score(pred_val_lr,y_val)

0.2431569431210462

In [ ]:
# Because we are doing document cassification we are supplyimg the untransposed tf_idf matrix

In [ ]:
# Viewing tf_idf matrix is difficult becasue it is a saprse matrix and it is stored in a difficult format.
# For viewing it we need to convert it to dense format and then view it.
# Tf idf is stored as a row ,coloumn and value .It is called a triplet matrix.This format doent store the entire thing,
# means doesnt store the0s.We just store the indexes for which the values are non-zero 

In [37]:
# We take N=150
# 150<3657 since we are truncating.

In [ ]:
# tfidf_mat_train is a document vector.We represented each and every word by a sequence of 150 numbers

In [22]:
N=150
svd=TruncatedSVD(n_components=N)
svd_mat_train=svd.fit_transform(tfidf_mat_train)
svd_mat_val=svd.transform(tfidf_mat_val)
svd_mat_test=svd.transform(tfidf_mat_test)

In [23]:
lr_model_2=LogisticRegression()
lr_model_2.fit(X=svd_mat_train,y=y_train)
pred_val_lr2=lr_model_2.predict(svd_mat_val)
cohen_kappa_score(pred_val_lr2,y_val)

0.2449596562952483

In [24]:
gbm=GradientBoostingClassifier(learning_rate=0.01,n_estimators=2000,max_depth=1)

In [25]:
gbm_1=gbm.fit(X=svd_mat_train,y=y_train)
pred_val_gbm1=gbm_1.predict(svd_mat_val)
cohen_kappa_score(pred_val_gbm1,y_val)

0.23983560652579927

In [38]:
# 10 class classification problem.Getting high cohen kappa will really be difficult.
# Instead of classification problem consider it as a regression problem and do it
# Discretize the values of the y_test to only 2 or 3 values.Above 0.7 suppose posoitive.4-6 is neutral.1-3 is negative.

In [39]:
# Regressionwas avoided and if we used it then score could have gone beyond 10.??Where we would have to use clipping??
# Differentiating betweena 10 and 9 is difficult.So if we discretize it and lower no of classes it will be much more rational.And we can 
# Potentially significant improvement in our model.As cohen_kappa_score goes significantly beyond 0.5 and 0.4 we can say that my
# model is good enough for prediction. 

In [40]:
# Whatever we have done till nbow is in the reviewer's perspective.if we wanna create the word vector the steps are shown below.

In [26]:
svd_new=TruncatedSVD(n_components=150)

In [27]:
word_vec=svd_new.fit_transform(tfidf_mat_train.T)

In [44]:
# In original the formats are documents are in rows and words in columns

In [45]:
# In word_vec words are thus in rows and documents are in columns

In [28]:
word_vec.shape

(3657, 150)

In [ ]:
# Since we have used tfidf_mat_train.T above thus we are working with the word_vec here.
# In word vector each word is being represented by some numeric or floating point values.So suppose avengers it is being
# defined by 150 floating points 

In [29]:
word_vec

array([[ 0.16663287, -0.02542924, -0.02796946, ...,  0.0014191 ,
        -0.11207288,  0.03105973],
       [ 0.07443251, -0.05981533, -0.0684805 , ..., -0.03803257,
        -0.05792062, -0.0279033 ],
       [ 0.14860485,  0.00560973,  0.03498279, ..., -0.02677417,
        -0.05381894, -0.00472174],
       ...,
       [ 0.12356809, -0.06661412, -0.07678172, ...,  0.0313599 ,
         0.01704897, -0.03999437],
       [ 0.05873907, -0.03514341, -0.04228364, ...,  0.02167834,
         0.01068513, -0.01001146],
       [ 0.02800249, -0.0145984 , -0.00817314, ..., -0.00589331,
         0.01238936, -0.00365233]])

In [46]:
tfidf.vocabulary_

{'will': 3580,
 'win': 3582,
 'oscar': 2275,
 'worst': 3624,
 'scenario': 2792,
 'director': 881,
 'godfather': 1389,
 'probably': 2481,
 'perfect': 2346,
 'made': 1956,
 'strong': 3103,
 'plot': 2412,
 'unique': 3430,
 'cast': 454,
 'direction': 879,
 'filming': 1232,
 'editing': 986,
 'beyond': 297,
 'words': 3610,
 'music': 2136,
 'makes': 1968,
 'blend': 315,
 'great': 1418,
 'movies': 2123,
 'course': 693,
 'lack': 1795,
 'one': 2254,
 'two': 3397,
 'things': 3269,
 'mentioned': 2047,
 'difficult': 866,
 'match': 2006,
 'francis': 1311,
 'coppola': 668,
 'part': 2324,
 'ii': 1588,
 'may': 2014,
 'masterpiece': 2004,
 'watch': 3538,
 'never': 2176,
 'bored': 342,
 'always': 99,
 'surprised': 3177,
 'wonderful': 3604,
 'see': 2828,
 'young': 3648,
 'al': 76,
 'pacino': 2297,
 'change': 483,
 'college': 572,
 'boy': 355,
 'ruthless': 2751,
 'making': 1969,
 'even': 1068,
 'marlon': 1994,
 'brando': 362,
 'title': 3317,
 'role': 2721,
 'albeit': 77,
 'small': 2955,
 'actor': 33,
 'rob

In [30]:
tfidf.vocabulary_['hulk']

1562

In [47]:
# Lets find out words which are similar to Hulk

In [31]:
from sklearn.metrics import pairwise_distances

In [32]:
dist_hulk=pairwise_distances(X=word_vec,Y=word_vec[1562,:].reshape(1,-1),metric='cosine')

In [ ]:
# ??If reshape is 1,-1 row=1 and coloumn=150??

In [ ]:
# ??Checkout the above reshaping thing??

In [33]:
dist_hulk.flatten().argsort() #why flattened?

array([1562, 3275, 1480, ..., 2696, 1220, 2213])

In [34]:
id_to_word={v:k for k,v in tfidf.vocabulary_.items()}

In [50]:
id_to_word[1699]

'iron'

In [49]:
 id_to_word[3275]

'thor'

In [ ]:
# If n gram (1,2) single and double word was taken then we could have got more interesting things like iron man.